Lo mismo que el otro notebook pero por forks

In [1]:
%run ../../data/api_keys.ipynb
%run ../modulos.ipynb
%run ../funciones_temp.ipynb

In [3]:
def request_github(url,personal_access=personal_access_token):
    headers = {"Authorization": f"Bearer {personal_access}"}

    res=requests.get(url,headers=headers)
    return res.json()

In [4]:
def get_pages_repos(min_estrellas, max_estrellas, num_pagina=1):
    return f'https://api.github.com/search/repositories?per_page=100&page={num_pagina}&q=forks%3A{min_estrellas}..{max_estrellas}&sort=stars&order=asc'

In [7]:
get_pages_repos(30,60)

'https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A30..60&sort=stars&order=asc'

In [5]:
headers = {"Authorization": f"Bearer {personal_access_token}","Accept": "application/vnd.github.v3.star+json","content-type": "application/json"}

In [6]:
client=pymongo.MongoClient('mongodb://localhost:27017/')
database=client['promisearch'] # database
collecion_repos_forks=database['repositorios_forks'] # colección
len(list(collecion_repos_forks.find()))

0

## Inserciones a DB

In [8]:
def obtener_hist(user,repo,atributo,nom_col):

    consulta_url = f"https://api.github.com/repos/{user}/{repo}/{atributo}"
    repos_resp = []
    
    page=1
    response_no_json = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page))
    response=response_no_json.json() # primeras 100
    
    # Estos try son para cuando excedes el límite de 5000 por hora 
    try:
        repos_resp.extend([re[nom_col] for re in response])
    except:
        print('Número máximo alcanzado, esperando 30s')
        time.sleep(30)
        page=page-1
        

    while len(response)!=0:
        page+=1
        response = request_github_no_json(consulta_url+"?per_page=100"+"&page="+str(page)).json()
        
        try:
            repos_resp.extend([re[nom_col] for re in response])
        except:
            print('Número máximo alcanzado, esperando 30s')
            time.sleep(30)
            page=page-1
            
            

  
    serie_acum=lista_fecha_a_df(repos_resp)
    try:
        cuantiles=pd.qcut(serie_acum, q=4, retbins=True)[1]
        cuant_prop=f'{cuantiles[1]},{",".join([str(round(ai,4)) for ai in list(cuantiles[2:] / cuantiles[1:-1])])}'
    except:
        return '0,0,0,0'

    return cuant_prop

Límite de '5,000 requests per hour'

In [9]:
def obtener_info_repo(user,repo):
    """Regresa json con info histórica y del repo"""

    star_count_url = "https://api.github.com/repos/"+user+"/"+repo

    response1 = request_github_no_json(star_count_url)
     
    cuantiles_estrellas = obtener_hist(user,repo,'stargazers','starred_at')
    cuantiles_forks = obtener_hist(user,repo,'forks','created_at')

    resp_json=response1.json()
    resp_json['cuant_estrellas']=cuantiles_estrellas
    resp_json['cuant_forks']=cuantiles_forks
 
    return resp_json

In [10]:
# Prueba de que todo está funcionando
res_json=obtener_info_repo('pysentimiento','pysentimiento')
print(res_json.get('id',np.nan))
print(res_json.get('html_url',np.nan))
print(res_json.get('cuant_estrellas',np.nan))
print(res_json.get('cuant_forks',np.nan))
print(res_json.get('size',np.nan))
print(res_json.get('stargazers_count',np.nan))
print(res_json.get('forks',np.nan))
print(res_json.get('open_issues',np.nan))
print(res_json.get('subscribers_count',np.nan))
print(res_json.get('network_count',np.nan))

292896321
https://github.com/pysentimiento/pysentimiento
80.25,1.6386,1.2833,1.2741
7.5,1.7333,1.4231,1.2973
14289
215
24
4
7
24


In [12]:
def obtener_repos_de_pags(min_estrellas,max_estrellas,numero_requests,num_muestreo, k):
    repos_final = {}
    num_pagina=1 
    
    # Sólo agarrar muestreo de repos
    for _ in range(numero_requests):
        repos_bins = []

        i=0
        while i <1:
            # Construyo la URL
            url_objetivo = get_pages_repos(min_estrellas, max_estrellas, num_pagina)
            #print(num_pagina)
            print(url_objetivo)
            # Aqui hago la peticion a github
            search_github = requests.get(url_objetivo ,headers=headers)
            search_github_json = search_github.json()
            try:
                lista_repos = search_github_json['items'] # 100 repos
                lista_random = random.sample(lista_repos, num_muestreo)   
                print(f'Todo bien {k}')             
                repos_bins.extend(lista_random)
                
            except:  
                print('A dormir')
                time.sleep(61)
                i-=1
            i+=1
        num_pagina += 1
        
        repos_final[f'{_}'] = repos_bins
        collecion_repos_forks.insert_one({f'{k}':repos_bins})
    return repos_final

In [24]:
# collecion_repos.drop()

In [13]:
# Esto es para obtener repos al azar de diferentes páginas
num_paginas_visitar=1
num_repos_por_pagina=5

#------Memo----------- 50 
# for k in range(1, 10):
# Avitua
# # for k in range(10, 1000000000):
start=1
stop=100
for k in range(start, stop):
    obtener_repos_de_pags(30*k,30*(k+1),num_paginas_visitar,num_repos_por_pagina, k)

https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A30..60&sort=stars&order=asc
Todo bien 1
https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A60..90&sort=stars&order=asc
Todo bien 2
https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A90..120&sort=stars&order=asc
Todo bien 3
https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A120..150&sort=stars&order=asc
Todo bien 4
https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A150..180&sort=stars&order=asc
Todo bien 5
https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A180..210&sort=stars&order=asc
Todo bien 6
https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A210..240&sort=stars&order=asc
Todo bien 7
https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A240..270&sort=stars&order=asc
Todo bien 8
https://api.github.com/search/repositories?per_page=100&page=1&q=forks%3A270..300&sor

In [15]:
s=set()
for c in list(collecion_repos_forks.find()):
  s.add(list(c.keys())[1])
  
a=[]
for si in s:
  a.append(int(si))
a=np.array(a)

a.sort()
a

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

## Creación de csv

In [22]:
id_repos = []
name_repos = []
sizes = []
stargazers_count_list = []
forks_list = []
open_issues_list = []
watchers = []
network_count_list = []
suscribers_list = []
repos1=[]

cuantiles_star=[]
cuantiles_forks=[]
lenguajes=[]

# comienzo final dice en que rango de la base de datos se van a buscar los datos de los repos
comienzo=50
final=60
for k in tqdm_notebook(range(comienzo,final)):
    lista_dict=list(collecion_repos_forks.find({ f'{k}' : { '$exists' : 'true' } }))[0][f'{k}']
    #lista_dict=repos_final[f'{k}']
    for repo in tqdm_notebook(lista_dict, desc='Inner loop'):
        repo_name=repo['name']
        user=repo['owner']['login']
        # --------------------------------------------------
        json_repo=obtener_info_repo(user,repo_name)

        id_repos.append(json_repo.get('id',np.nan))
        name_repos.append(json_repo.get('html_url',np.nan))
        cuantiles_star.append(json_repo.get('cuant_estrellas',np.nan))
        cuantiles_forks.append(json_repo.get('cuant_forks',np.nan))
        lenguajes.append(json_repo.get('language',np.nan))
        sizes.append(json_repo.get('size',np.nan))
        stargazers_count_list.append(json_repo.get('stargazers_count',np.nan))
        forks_list.append(json_repo.get('forks',np.nan))
        open_issues_list.append(json_repo.get('open_issues',np.nan))
        watchers.append(json_repo.get('subscribers_count',np.nan))
        network_count_list.append(json_repo.get('network_count',np.nan)) # número de personas que han "contribuido"

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:22: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

...

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

Inner loop:   0%|          | 0/5 [00:00<?, ?it/s]

In [23]:
repos_df = pd.DataFrame(dict(
    ids=id_repos,
    name_repo=name_repos,
    lenguaje=lenguajes,
    stargazers=stargazers_count_list,
    watchers=watchers,
    cuantiles_star=cuantiles_star,
    cuantiles_forks=cuantiles_forks,
    forks=forks_list,
    # used_by=[],   
    tamaño_kb=sizes,
    issues=open_issues_list,
    network_size=network_count_list
    # pull_request=[],
    # branches=[],
    # no_commits=[]
))

repos_df

,ids,name_repo,lenguaje,stargazers,watchers,cuantiles_star,cuantiles_forks,forks,tamaño_kb,issues,network_size
0,45498993,https://github.com/learn-co-students/nested-ha...,Ruby,0,10,"0,0,0,0","327.5,2.0702,1.6777,1.3152",1497,27,1641,1497
1,46398090,https://github.com/datahub-project/datahub,Java,5519,222,"778.5,1.8773,1.987,1.9005","300.0,1.8833,1.6053,1.645",1512,481983,242,1512
2,48230104,https://github.com/progedu/intro-curriculum-3011,JavaScript,0,3,"0,0,0,0","303.0,1.9736,1.9933,1.2659",1509,3,1529,1509
3,55850689,https://github.com/open-falcon/falcon-plus,Go,6905,416,"2328.5,1.9268,1.3445,1.1447","412.5,2.0582,1.4217,1.2469",1522,17291,163,1522
4,287464616,https://github.com/migueravila/Bento,JavaScript,1141,11,"232.5,2.1978,1.5098,1.4789","425.0,1.78,1.4124,1.4207",1555,14734,12,1555
5,10247874,https://github.com/docker/docker-py,Python,5764,187,"1131.5,2.274,1.7184,1.3036","362.25,2.0897,1.4937,1.3266",1549,5200,515,1549
6,21246279,https://github.com/googlearchive/android-Camer...,Java,2789,164,"581.5,2.3207,1.6612,1.2441","358.75,2.0697,1.5949,1.2759",1548,1453,0,1548
7,137503442,https://github.com/void-linux/void-packages,Shell,1706,51,"589.5,1.626,1.3495,1.3189","411.75,1.8883,1.4421,1.3387",1535,511797,1394,1535
8,45213473,https://github.com/learn-co-students/method-ar...,Ruby,0,10,"0,0,0,0","324.25,2.0262,1.7272,1.2998",1536,14,1621,1536
9,52710065,https://github.com/all-contributors/all-contri...,HTML,6273,67,"1247.5,2.489,1.5736,1.2839","286.0,2.3007,1.6945,1.3641",1537,17343,94,1537


In [24]:
repos_df[['star_q1','star_gq2','star_gq3','star_gq4']]=repos_df.cuantiles_star.str.split(',',expand=True) 
repos_df.drop(columns=['cuantiles_star'],inplace=True)
repos_df[['fork_q1','fork_gq2','fork_gq3','fork_gq4']]=repos_df.cuantiles_forks.str.split(',',expand=True) 
repos_df.drop(columns=['cuantiles_forks'],inplace=True)
repos_df.tail()

,ids,name_repo,lenguaje,stargazers,watchers,forks,tamaño_kb,issues,network_size,star_q1,star_gq2,star_gq3,star_gq4,fork_q1,fork_gq2,fork_gq3,fork_gq4
45,399067972,https://github.com/AliyunWorkbench/hexo,HTML,16,1,1784,969,5,1784,4.75,1.7895,1.4412,1.3061,433.0,2.351,1.6999,1.0303
46,47057247,https://github.com/Wind4/vlmcsd,C,6177,305,1793,12611,6,1793,1069.0,2.8213,1.5729,1.3021,383.0,2.2219,1.5746,1.3321
47,86971730,https://github.com/huihut/CS-Books,None,2964,88,1784,1590520,4,1784,856.75,2.5194,1.2206,1.125,514.0,2.177,1.2542,1.1407
48,22749672,https://github.com/JerryLead/SparkInternals,None,4820,618,1780,70752,28,1780,1542.0,1.7419,1.4505,1.2372,539.5,1.8026,1.4075,1.261
49,31102092,https://github.com/google/or-tools,C++,8216,272,1784,1267868,64,1784,1571.5,2.223,1.7194,1.3678,395.5,2.091,1.5677,1.3536


In [25]:
print(repos_df.stargazers.isna().sum())
print(repos_df.forks.isna().sum())
repos_df.forks.describe()

0
0


count      50.000000
mean     1667.220000
std       120.167789
min      1497.000000
25%      1594.750000
50%      1656.500000
75%      1739.750000
max      2209.000000
Name: forks, dtype: float64

In [26]:
print(comienzo*30,final*30)

1500 1800


In [27]:
repos_df.to_csv(f'../../data/data_sin_etiqueta_jala_fork{comienzo*30}-{final*30}.csv', index=False)